## Crear una instancia EC2, ejecutarla, pararla y eliminarla

In [2]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv()

aws_access_key = os.getenv("aws_access_key_id")
aws_secret_key = os.getenv("aws_secret_access_key")
aws_session_token = os.getenv("aws_session_token")
region = os.getenv("aws_default_region")

ec2_res = boto3.resource(
    "ec2",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region
)

ec2_cli = boto3.client(
    "ec2",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region,
)

def obtener_estado_instancia(id):
    response = ec2_cli.describe_instances(InstanceIds=[id])
    state = response["Reservations"][0]["Instances"][0]["State"]["Name"]
    print(f"Estado actual de {id}: {state}")

#### Crear instancia

In [3]:
instances = ec2_res.create_instances(
    ImageId="ami-08b5b3a93ed654d19",
    InstanceType="t2.micro",
    MinCount=1,
    MaxCount=1,
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "InstanciaBoto3"}],
        }
    ],
)

instance = instances[0]

print(f"id instancia: {instance.id}")

id instancia: i-03c48d90e51224fcb


#### Parar la instancia

In [6]:
response = ec2_cli.stop_instances(InstanceIds=[instance.id])
print(response)

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-03c48d90e51224fcb', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '3a5d3ce6-036d-4933-a1a1-63b60a477063', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3a5d3ce6-036d-4933-a1a1-63b60a477063', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '411', 'date': 'Sat, 15 Mar 2025 15:30:02 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [7]:
obtener_estado_instancia(instance.id)

Estado actual de i-03c48d90e51224fcb: stopping


#### Eliminar instancia

In [8]:
response = ec2_cli.terminate_instances(InstanceIds=[instance.id])
state = response["TerminatingInstances"][0]["CurrentState"]["Name"]
print(f"Estado: {state}")

Estado: shutting-down


## Cerar un EBS y asociarlo a un EC2 y añadir una archivo

In [9]:
instances = ec2_res.create_instances(
    ImageId="ami-08b5b3a93ed654d19",
    InstanceType="t2.micro",
    MinCount=1,
    MaxCount=1,
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "InstanciaBoto3"}],
        }
    ],
)

instance = instances[0]

print(f"id instancia: {instance.id}")

id instancia: i-060d20dcf8942a636


#### Crear y asociar

In [10]:
response = ec2_cli.describe_instances(InstanceIds=[instance.id])
availability_zone = response["Reservations"][0]["Instances"][0]["Placement"]["AvailabilityZone"]

In [11]:
volume_response = ec2_cli.create_volume(
    AvailabilityZone=availability_zone,
    Size=8,
    VolumeType="gp2",
    TagSpecifications=[
        {
            "ResourceType": "volume",
            "Tags": [{"Key": "Name", "Value": "miEBS"}],
        }
    ],
)

print(f"id ebs: {volume_response["VolumeId"]}")

id ebs: vol-0272207ce16fd5221


In [15]:
vol_status = ec2_cli.describe_volumes(VolumeIds=[volume_response["VolumeId"]])["Volumes"][0]["State"]
if (vol_status == "available"):
    ec2_cli.attach_volume(
        VolumeId=volume_response["VolumeId"],
        InstanceId=instance.id,
        Device="/dev/sdf"
    )
    print("vinculado")
elif (vol_status == "in-use"):
    print("ya asociado")
else:
    print("ebs no esta disponible aun")

vinculado


#### Anadir archivo (ssh)

In [16]:
response = ec2_cli.describe_instances(InstanceIds=[instance.id])
EC2_PUBLIC_IP = response["Reservations"][0]["Instances"][0]["PublicIpAddress"]
print(EC2_PUBLIC_IP)

SSH_KEY_PATH = "labsuser.pem"
EC2_USER = "ec2-user"

VOLUME_DEVICE = "/dev/xvdf"
MOUNT_POINT = "/mnt/ebs"
LOCAL_FILE_PATH = "skere.txt"  

34.236.150.180


In [14]:
# import paramiko

# ssh = paramiko.SSHClient()
# ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# ssh.connect(EC2_PUBLIC_IP, username=EC2_USER, key_filename=SSH_KEY_PATH)

## Crear un EFS, montarlo y añadir un archivo

#### Crear efs

In [17]:
efs_client = boto3.client(
    "efs",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region
)

response = efs_client.create_file_system(
    PerformanceMode="generalPurpose",
    ThroughputMode="bursting",
    Tags=[{"Key": "Name", "Value": "miEFS"}],
)

EFS_ID = response["FileSystemId"]
print(f"id efs: {EFS_ID}")


id efs: fs-01fdd7cba6b6f4bb2


#### Montaje

In [18]:
# Obtener la VPC y una subred disponible
vpcs = ec2_cli.describe_vpcs()
vpc_id = vpcs["Vpcs"][0]["VpcId"]

subnets = ec2_cli.describe_subnets(Filters=[{"Name": "vpc-id", "Values": [vpc_id]}])
subnet_id = subnets["Subnets"][0]["SubnetId"]

# Crear punto de acceso para el EFS en la subred
response = efs_client.create_mount_target(
    FileSystemId=EFS_ID,
    SubnetId=subnet_id,
    SecurityGroups=["sg-0e4ebccba24ef9c17"],
)

print(f"Punto de montaje en subred {subnet_id}")

Punto de montaje en subred subnet-0873db01c53d03f44


In [20]:
print(instance.id)

i-060d20dcf8942a636


In [21]:
import time

ssm_client = boto3.client("ssm", region_name=os.getenv("AWS_DEFAULT_REGION"))

MOUNT_POINT = "/mnt/efs"

# Comandos para instalar el cliente NFS y montar el EFS
commands = [
    "sudo yum install -y amazon-efs-utils",
    f"sudo mkdir -p {MOUNT_POINT}",
    f"sudo mount -t efs {EFS_ID}:/ {MOUNT_POINT}",
    f"echo '{EFS_ID}:/ {MOUNT_POINT} efs defaults,_netdev 0 0' | sudo tee -a /etc/fstab",
]

# Ejecutar el comando en EC2 con AWS SSM
response = ssm_client.send_command(
    InstanceIds=[instance.id],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": commands},
)

# Esperar a que se complete el montaje
command_id = response["Command"]["CommandId"]
print(f"✅ Comandos enviados para instalar y montar EFS. Command ID: {command_id}")

time.sleep(10)  # Esperar a que el proceso termine


InvalidInstanceId: An error occurred (InvalidInstanceId) when calling the SendCommand operation: Instances [[i-060d20dcf8942a636]] not in a valid state for account 932299691865

### 

In [ ]:
ssm_client = boto3.client("ssm", region_name="us-east-1")

response = ssm_client.describe_instance_information()

instances = [i["InstanceId"] for i in response["InstanceInformationList"]]

if instance.id in instances:
    print(f"✅ La instancia {instance.id} está registrada en SSM.")
else:
    print(f"❌ La instancia {instance.id} NO está registrada en SSM. Verificá el agente SSM e IAM.")


❌ La instancia i-0c4fff4915da26cc9 NO está registrada en SSM. Verificá el agente SSM e IAM.


: 

**Links**  
 [Crear y asociar ebs](https://stackoverflow.com/questions/61089756/using-boto3-to-create-and-attach-an-ebs-volume-to-an-ec2-instance)  
 [Usuarios EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/managing-users.html)